In [2]:
import keras
from keras.models import model_from_json
model_original = model_from_json(open("record_5_model_PQzyq_modelJson.json").read())
model_original.load_weights("record_5_model_PQzyq_modelWeights.h5")

In [3]:
model = model_from_json(open("tf_order_record_5_model_PQzyq_modelJson.json").read())

In [4]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 1, 190, 50)    2250        convolution2d_input_2[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 1, 190, 50)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 1, 180, 50)    27550       activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 1, 180, 50)    0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [5]:
from keras import backend as K
from keras.utils.np_utils import convert_kernel
import tensorflow as tf
import numpy as np
ops = []
#convert the conv layers
for old_layer, new_layer in zip(model_original.layers, model.layers):
    if old_layer.__class__.__name__ in ['Convolution2D', 'Convolution1D']:
        original_w, original_b = old_layer.get_weights()
        #transpose it; it's currently in outchan,inchan,rows,cols
        #make it rows,cols,inchan,outchan
        transposed_w = np.transpose(original_w[:,:,::-1,::-1], (2,3,1,0))
        new_layer.set_weights([transposed_w, original_b])

In [6]:
#convert the dense layer
dense_w, dense_b = model_original.layers[6].get_weights()
print(dense_w.shape)
reshaped_dense_w = np.reshape(dense_w, (50,1,1,50))
axes_swapped_dense_w = np.transpose(reshaped_dense_w,(1,2,0,3))
new_dense_w = np.reshape(axes_swapped_dense_w, (50,50))
model.layers[6].set_weights([new_dense_w, dense_b])

#second-to-last dense layer just has regular weights
model.layers[9].set_weights(model_original.layers[9].get_weights())

(50, 50)


In [7]:
model.save('tf_order_record_5_model_PQzyq_modelWeights.h5')

In [9]:
np.random.seed(1234)
vals = np.random.random((1,4,200,1))
print(model.predict([vals]))
from keras.models import Model
layer_model = Model(input=model.input, output=model.layers[9].output)
print(np.sum(layer_model.predict([vals])))
print(layer_model.predict([vals]))

[[ 0.19056688  0.39294827  0.29661816]]
-2.74472
[[-1.4463309  -0.43493608 -0.86345422]]
